In [1]:
# ruff: noqa: F401

In [2]:
%load_ext autoreload
%load_ext pyinstrument

%autoreload 2

In [3]:
import sys

sys.path.insert(0, "..")

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
from datetime import datetime, timedelta
from pathlib import Path

import hvplot.polars
import numpy as np
import polars as pl

In [ ]:
from utils import compare

from tsdb_benchmarks.dbs.common import drop_table
from tsdb_benchmarks.dbs.monetdb import MonetDB
from tsdb_benchmarks.dbs.monetdb.fetch import fetch_binary, fetch_pymonetdb
from tsdb_benchmarks.dbs.monetdb.insert import insert, upsert

db = MonetDB()

In [7]:
df = pl.read_parquet("../data/input/time_series/data_0.2M_0.5k.parquet").with_columns(
    (100 * pl.col.col_1 - 50).cast(pl.Int8).alias("col_1_int8"),
    (100 * pl.col.col_1 - 50).cast(pl.Int16).alias("col_1_int16"),
    (100 * pl.col.col_1 - 50).cast(pl.Int32).alias("col_1_int32"),
    (100 * pl.col.col_1 - 50).cast(pl.Int64).alias("col_1_int64"),
    pl.col.col_1.cast(pl.Float32).alias("col_1_float32"),
    pl.col.col_1.cast(pl.Float64).alias("col_1_float64"),
    pl.col.col_1.cast(pl.Decimal(10, 2)).alias("col_1_decimal_10_2"),
    pl.col.col_1.cast(pl.Decimal(18, 3)).alias("col_1_decimal_18_3"),
    pl.col.col_1.cast(pl.String).alias("col_1_str"),
    (pl.col.col_1 > 0.5).cast(pl.Boolean).alias("col_1_bool"),
    pl.col.col_1.cast(pl.Binary).alias("col_1_blob"),
    ('{ "val": ' + pl.col.col_1.cast(pl.String) + "}").str.json_decode().alias("col_1_json"),
    pl.col.time.cast(pl.Time).alias("time_time"),
    pl.col.time.cast(pl.Date).alias("time_date"),
)


df = df.with_columns(
    pl.when((pl.col.time > df.get_column("time")[4]) | (pl.col.time == df.get_column("time")[0]))
    .then(pl.col(n))
    .otherwise(None)
    for n in df.columns
    if n != "time"
)

# df = df.select([n for n in df.columns if n == "time" or n.count("_") == 2])

cutoff = 50
overlap = 50

df_first = df.with_row_index().filter(pl.col.index < len(df) - cutoff).drop("index")
df_second = df.with_row_index().filter(pl.col.index > len(df) - (cutoff + overlap)).drop("index")
df_second = df_second.with_columns(pl.selectors.float() * 1_000)

In [8]:
TABLE_NAME = "test_upsert"

In [9]:
drop_table(TABLE_NAME, db.connect())

/Users/williamlauren/repos/tsdb-benchmarks/.venv/lib/python3.12/site-packages/sqlalchemy_monetdb/__init__.py:5: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [10]:
insert(df_first, TABLE_NAME, db.connect(), primary_key="time")

In [11]:
fetch_binary(f"select time, col_125 from {TABLE_NAME} order by time", db.connect())

time,col_125
datetime[ms],f32
2024-08-15 02:41:00,0.617713
2024-08-15 02:42:00,null
2024-08-15 02:43:00,null
2024-08-15 02:44:00,null
2024-08-15 02:45:00,null
…,…
2024-12-31 23:06:00,0.446498
2024-12-31 23:07:00,0.250259
2024-12-31 23:08:00,0.203715


In [12]:
upsert(df_second, TABLE_NAME, db.connect(), primary_key="time")

In [13]:
# %%pyinstrument

for idx in range(len(df_second)):
    upsert(df_second[idx], TABLE_NAME, db.connect(), primary_key="time")

    print(fetch_pymonetdb(f"select max(time) from {TABLE_NAME}", db.connect()))

shape: (1, 1)
┌─────────────────────┐
│ %1                  │
│ ---                 │
│ datetime[ms]        │
╞═════════════════════╡
│ 2025-01-01 00:00:00 │
└─────────────────────┘
shape: (1, 1)
┌─────────────────────┐
│ %1                  │
│ ---                 │
│ datetime[ms]        │
╞═════════════════════╡
│ 2025-01-01 00:00:00 │
└─────────────────────┘
shape: (1, 1)
┌─────────────────────┐
│ %1                  │
│ ---                 │
│ datetime[ms]        │
╞═════════════════════╡
│ 2025-01-01 00:00:00 │
└─────────────────────┘
shape: (1, 1)
┌─────────────────────┐
│ %1                  │
│ ---                 │
│ datetime[ms]        │
╞═════════════════════╡
│ 2025-01-01 00:00:00 │
└─────────────────────┘
shape: (1, 1)
┌─────────────────────┐
│ %1                  │
│ ---                 │
│ datetime[ms]        │
╞═════════════════════╡
│ 2025-01-01 00:00:00 │
└─────────────────────┘
shape: (1, 1)
┌─────────────────────┐
│ %1                  │
│ ---                 │
│ da

In [14]:
dff = fetch_binary(f"select * from {TABLE_NAME} order by time", db.connect())

dff

time,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,…,col_478,col_479,col_480,col_481,col_482,col_483,col_484,col_485,col_486,col_487,col_488,col_489,col_490,col_491,col_492,col_493,col_494,col_495,col_496,col_497,col_498,col_499,col_500,col_1_int8,col_1_int16,col_1_int32,col_1_int64,col_1_float32,col_1_float64,col_1_decimal_10_2,col_1_decimal_18_3,col_1_str,col_1_bool,col_1_blob,col_1_json,time_time,time_date
datetime[ms],f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i16,i32,i64,f32,f64,"decimal[10,2]","decimal[18,3]",str,bool,binary,struct[1],time,date
2024-08-15 02:41:00,0.99439,0.655433,0.117288,0.11442,0.091383,0.797504,0.75722,0.527782,0.464873,0.592167,0.854792,0.210921,0.604031,0.081711,0.019506,0.706256,0.327759,0.218371,0.825249,0.884843,0.926574,0.718446,0.827058,0.980189,0.392664,0.436836,0.005624,0.752218,0.828335,0.480415,0.55338,0.802938,0.249983,0.703698,0.759349,0.504259,…,0.203039,0.13414,0.379596,0.519513,0.692039,0.268861,0.806501,0.765445,0.797289,0.784079,0.409881,0.406423,0.636099,0.759247,0.968955,0.231726,0.999709,0.412795,0.590418,0.337381,0.401983,0.837193,0.988874,49,49,49,49,0.99439,0.99439,0.99,0.994,"""0.99438983""",true,"b""0.99438983""",{0.99439},02:41:00,2024-08-15
2024-08-15 02:42:00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2024-08-15 02:43:00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2024-08-15 02:44:00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2024-08-15 02:45:00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-12-31 23:56:00,606.381531,833.705444,64.702682,451.500031,954.778259,524.600281,430.437042,218.519424,909.366577,756.413757,115.018074,848.976379,97.597397,295.432922,667.967773,346.527863,18.431116,84.524475,627.159424,616.916199,545.490234,448.649963,343.535217,393.813965,788.949829,889.93219,33.776928,632.749023,662.15094,821.839417,218.898453,280.356476,172.431427,485.55777,828.015381,174.793716,…,525.676331,330.433716,105.925049,633.961304,717.916626,631.813538,177.693283,804.182312,326.624268,561.434448,73.278053,974.752014,22.943497,282.813477,316.544403,191.472366,640.458679,167.137451,842.935486,281.186737,988.227356,358.5802,42.129215,10,10,10,10,606.381531,606.381536,0

In [15]:
compare(dff, df)

✅ Checking DataFrames...
❌ DataFrames are NOT equal.

✅ Column names match.

⚠️ Columns with differences: ['col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8', 'col_9', 'col_10', 'col_11', 'col_12', 'col_13', 'col_14', 'col_15', 'col_16', 'col_17', 'col_18', 'col_19', 'col_20', 'col_21', 'col_22', 'col_23', 'col_24', 'col_25', 'col_26', 'col_27', 'col_28', 'col_29', 'col_30', 'col_31', 'col_32', 'col_33', 'col_34', 'col_35', 'col_36', 'col_37', 'col_38', 'col_39', 'col_40', 'col_41', 'col_42', 'col_43', 'col_44', 'col_45', 'col_46', 'col_47', 'col_48', 'col_49', 'col_50', 'col_51', 'col_52', 'col_53', 'col_54', 'col_55', 'col_56', 'col_57', 'col_58', 'col_59', 'col_60', 'col_61', 'col_62', 'col_63', 'col_64', 'col_65', 'col_66', 'col_67', 'col_68', 'col_69', 'col_70', 'col_71', 'col_72', 'col_73', 'col_74', 'col_75', 'col_76', 'col_77', 'col_78', 'col_79', 'col_80', 'col_81', 'col_82', 'col_83', 'col_84', 'col_85', 'col_86', 'col_87', 'col_88', 'col_89', 'col_90', 'c